In [1]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [2]:
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer

In [4]:
import pandas as pd
import numpy as np

In [5]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/test.csv')

In [6]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
print(train_data.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [8]:
train_data['Street'].value_counts()

Pave    1454
Grvl       6
Name: Street, dtype: int64

In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [10]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


#подготовка датафреймов

In [11]:
def deleteId(ds):
    return ds.drop(columns='Id')

def deleteNaN(train_ds, test_ds, critval):
    fullsize = train_ds.shape[0]
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    for feature in new_train.columns:
        nulls = new_train[feature].isnull().sum()
        percent = nulls / fullsize
        # если доля пустых значений превышает critval - столбец не информативен,
        # можно его выбросить
        if (percent > critval):
            new_train = new_train.drop(columns=feature)
            new_test = new_test.drop(columns=feature)
    return new_train, new_test

def convertToNumeric(train_ds, test_ds):
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    LE = LabelEncoder()
    for feature in new_train.columns[:-1]:
        if (new_train[feature].dtype == 'object'):
            new_train[feature] = LE.fit_transform(new_train[feature])
            new_test[feature] = LE.fit_transform(new_test[feature])
    return new_train, new_test

train = train_data.drop_duplicates()
train = train.drop(columns='Id')
test = test_data.drop(columns='Id')

train, test = deleteNaN(train, test, critval=0.7)
train, test = convertToNumeric(train, test)

train_y = train['SalePrice'].values
train_X = train.drop(columns='SalePrice').values
val_test = test.values

train_x, test_x, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=98987)

Хорошо, но train остались пропущеные значения. Как их заполнить? 
Есть несколько тсратегий для заполнения пропущенных значений.

Заполнять пропущенные значения тем, что понравится на угад - не лучшая стратегия, часто применяют статистику для вычисления более-менее подходящего значения.

1. Самый простой и интуитивный способ - заполнить средним арифметическим значением колонки. Во-первых это не изменит среднее значение по всем значениям колонки.
2. Но иногда попадаются выбросы, поэтому, чтобы сделать вычисление среднего более робастным, можно брать усеченное среднее - сортируем значения в колонке, отбрасываем n штук слева и n штук справа и считаем среднее арифметическое.
3. Медианное среднее - медиана это точка ровно по середине выборки - сортируем значения колонки и тыкаем в середку. Такое среднее более устойчиво к выбросам, чем среднее арифметическое.
4. Наиболее часто встрчаемое - еще одна стратегия для заполнения - если чаще всего встречается - почему бы и нет.

In [12]:
strategies = ['mean', 'median', 'most_frequent']
imputer = SimpleImputer(strategy=strategies[1])
trainX = imputer.fit_transform(train_x)
testX = imputer.fit_transform(test_x)

In [13]:
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

In [14]:
trainX.shape

(1314, 75)

#RandomForest

In [15]:
parameters = {
    'criterion':(['squared_error']), 
    'max_depth':  range(1000, 3001, 500),
    'n_estimators': range(10, 51, 10),
    'min_samples_leaf': range(1, 9, 2),
    'min_samples_split': range(2, 11, 2)}

model = RandomForestRegressor()
gs_model = GridSearchCV(model, parameters)
gs_model.fit(trainX[:600], train_y[:600])
print(f"Best parameters: {gs_model.best_params_}")

Best parameters: {'criterion': 'squared_error', 'max_depth': 3000, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 20}


In [16]:
def check_error(preds, gt):
    print('RMSE Error:', mean_squared_error(np.log(preds), np.log(gt), squared=False))

In [17]:
model = RandomForestRegressor(**gs_model.best_params_)
model.fit(trainX, train_y)
check_error(model.predict(testX), test_y)

RMSE Error: 0.1663834410331219


#XGBoost

Бустинг - метод ансамблирования, в котором обучается много копий более слабой модели ("weak learner"), т.е. такой модели, которая не может достичь высокой точности на обучающей выборке, переобучившись на ней. Например, решающее дерево небольшой глубины. На каждом шаге новая модель концентрируется на исправлении ошибок предыдущей модели. Итоговое предсказание получается как взвешенное голосование всех моделей. Таким образом, главное отличие Бустинга от бэггинга - обучение моделей происходит зависимо от предыдущих моделей и последовательно, когда в бэггинге обучение происходит независимо и параллельно.

XGBoost - вычислительно эффективная реализация градиентного бустинга над решающими деревьями.

Суть градинетного бустинга в следующем: каждый последующий "слабый" алгоритм должен корректировать предсказания предыдущего. При этом входными параметрами следующего алгоритма является минус градиент функции потерь по предсказаниям предыдущего алгоритма. При этом от функции потерь не трубуется дифференцируемость по параметрам в отличе от градиентного спуска, но требуется диференцируемость по предсказаниям.

Пусть:
- $\{x_i, y_i\}_{i=1}^N$ - обучающая выборка
- K - кол-во деревьев в ансамбле
- $f_k:x \rightarrow y$ - k-e дерево ансамбля как функция
- $F: x → y$ - весь ансамбль как функция
- $loss: y_{true} \times y_{pred} → R$ - какая-то функция потерь
- $T_k$ - кол-во листьев в к-ом дереве
- $w_k$ - вектор, составленный из выходных значений на всех листьях к-го дерева

в XGBoost ответы суммируются по всем деревьям ансамбля: $$F(x) = Σ_{k=1}^Kf_x(x)$$

Суммарная функция потерь выглядит так: $$Loss_{total} = Σ_{i=1}^Nloss(y_i, F(x_i)) + γΣ_{k=1}^KT_k + \frac{1}{2}λΣ_{k=1}^K||w_k||^2,$$
где γ, λ - гиперпараметры.

Первое слагаемое - суммируем все потери по всем деревьям,

Второе слагаемое - штрафует деревья за слишком большое кол-во листьев,

Третье слагаемое - обеспечивает механизм, чтобы каждое дерево вносило минимальный вклад в результат.

Для минимизации $Loss_{total}$ используются первая и вторая производные по предсказаниями.

**Построение деревьев**

Введем целевую функцию (objective function): $$obj = Σ_{i=1}^Nloss(y_i, y_i^{'(t)} + Σ_{i=1}^tw(f_i) $$

Каждое следующее дерево должно улучшать предсказание предыдущего:
$$y_i^{'(t+1)} = y_i^{'(t)} + f_{t+1}(x_i)$$

Вопрос: как строить дереве на каждой следующей итерации? Интеитивный ответ - строить так, чтобы оптимизировать целевую функцию:

$obj^{(t)} = Σ_{i=1}^Nloss(y_i, y_i^{'(t)}) + Σ_{i=1}^tw(f_i) = Σ_{i=1}^Nloss(y_i, y_i^{'(t-1)} + f_t(x_i)) + Σ_{i=1}^tw(f_i) + constant$

если, например, использовать MSE:

$obj^{(t)} = Σ_{i=1}^N(y_i - ( y_i^{'(t-1)} + f_t(x_i)))^2 + Σ_{i=1}^tw(f_i) = Σ_{i=1}^N[2(y_i^{'(t-1)} - y_i)f_t(x_i) + f_t(x_i)^2] + Σ_{i=1}^tw(f_i) + constant$

в случае MSE получается полегче т.к. сама функция квадратичная, а основание линейное. В случае других функций потерь пользуемся разложением в ряд Тейлора.

$obj^{(t)} = Σ_{i=1}^N[loss(y_i, y_i^{'(t-1)}) + g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i)] + w(f_t) + const$, где:

$$g_i = \frac{dloss(y_i, y_i^{'(t-1)})}{dy_i^{'(t-1)}}$$
$$h_i = \frac{d^2loss(y_i, y_i^{'(t-1)})}{d(y_i^{'(t-1)})^2}$$

После приведения всех слогаемых, получаем оптимизационную задачу для построения нового дерева:

$$Σ_{i=1}^N[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)] + w(f_t)$$

Сложность будем определять следующим образом:

$$w(f) = γT + \frac{1}{2}λΣ_{j=1}^Tw_j^2$$, где w - вектор значений листьев, T - кол-во листьев

Тогда с учетом вышеуказанной регуляризации перепишем нашу целевую функцию:

$$obj^{(t)} = Σ_{i=1}^N[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)] + γT + \frac{1}{2}λΣ_{j=1}^Tw_j^2 = Σ_{j=1}^T[(Σ_{i из I_j}g_i)w_i + \frac{1}{2}(Σ_{i из I_j}h_i + λ)w_j^2] + γT$$, где $I_j = \{i|q(x_i)=j\}$ - множество элементов выборки, относящихся к j-ому листу.


положим $G_i =Σ_{i из I_j}g_i $ и $H_j=Σ_{i из I_j}h_i$, тогда

$$obj^{(t)} = Σ_{j=1}^T[G_jw_j + 1/2(H_j + λ)w_j^2] + γT$$ - получили функцию, которая квадратично зависит от $w_j$ - поэтому можем легко найти наилучшую оценку:

$$w_j^* = -\frac{G_j}{H_j + λ}$$
$$obj^* = -\frac{1}{2}Σ_{j=1}^T\frac{G_i^2}{H_j+λ}+γT$$ - измеряет, насколько построенное дерево "хорошее"

$$IG = \frac{1}{2}[\frac{G_L^2}{H_L+λ} + \frac{G_R^2}{H_R+λ} - \frac{(G_L + G_R)^2}{H_L+H_R+λ}] - γ$$

- первое слагаемое показывает оценку левого листа
- второе слагаемое оценку правого листа
- третье слагаемое оценка исходного листа
- четвертое слагаемое - регуляризация добавленного листа

Заметим, если прирост меньше чем регуляризация - мы не будем делить лист на две части.

Одним из важных настраиваемых параметров является **subsample** - которые указывает размер подвыборки но без возврата, и learning rate конечно.

In [21]:
# parameters = {
#     "learning_rate": (0.1, 0.01, 0.001),
#     "max_depth": range(3, 10, 1),
#     "min_child_weight": range(1, 11, 2),
#     "n_estimators": range(10, 91, 20),
#     "subsample": (0.5, 0.63, 0.7, 0.81)
# }

parameters = {
    "learning_rate": np.linspace(0.1, 0.5, 10),
    "max_depth": range(3, 10, 1),
    "min_child_weight": range(5, 16, 2),
    "n_estimators": range(30, 91, 10),
    "subsample": (0.63, 0.7, 0.75, 0.81)
}

model = XGBRegressor()
gs_model = GridSearchCV(model, parameters)
gs_model.fit(trainX[:600], train_y[:600])
print(f"Best parameters: {gs_model.best_params_}")

Выходные данные были обрезаны до нескольких последних строк (5000).
[07:30:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:30:29] WARNING: /workspace/src/objectiv

In [22]:
last_best_params = {
    "learning_rate": 0.1,
    "max_depth": 6,
    "min_child_weight": 9,
    "n_estimators": 50, 
    "subsample": 0.75
}

model = XGBRegressor(**gs_model.best_params_)
model.fit(trainX, train_y)
check_error(model.predict(testX), test_y)

[07:41:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE Error: 0.14739506007786296


#CatBoost

CatBoost - реализация градиентного бустинга над решающими деревьями.

Основные особенности:
- Использование решающих таблиц
- Упорядоченное target-кодирование на категориальных признаках высокой размерности
- Бустинг с упорядочиванием обучающих примеров

*target-кодирование* - это когда мы вместо категориального признака ставим некоторое статистически оцененное значение целевого признака, например, вместо марки машины - среднюю цену всех авто.

Решающая таблица - частный случай забывчивого дерева - все решающие правила одного уровня (на одинаковом расстоянии от корня) проверяют один и тот же признак. В CatBoost - на каждом уровне решающего дерева используется не только общий признак, но и общий порог разделения.

#LightGBM